______
# Natural Language Processing in Finance
______
When building an NLP pipeline for finance, high-quality textual data such as financial news, earnings call transcipts and analyst commentary is required to feed into models. Below are said accessible sources:

## Free Earnings Call Transcripts
- MarketBeat: free transcripts for S&P 500 earnings calls, available publicly via their website
- Seeking Alpha: comprehensive transcripts including slides and audio (may require sign up)
- Quartr: mobile app offering live and recorded earnings call transcripts with global coverage

## Financial News and Commentary APIs
- MarketAux: free tier API for real-time financial news across 5,000 sources
- FinnHub: free real-time news, along with market data like fundamentals and forex
- FinancialNewsAPI: aggregates millions of financial news articles with free access
- Financial Modelling Prep: free plan for stock news API
- StockNewsAPI.com: free endpoint for aggregated stock market news

## Datasets for Research and Model Training
- NIFTY Financial News Headlines Dataset: Contains 15.7M time aligned headlines and stock prices for 4,775 S&P 500 firms from 1999-2023, available on Hugging face
- FNSPIDL: dataset with 29.7M stock prices and 15.7M financial news records, ideal for sentiment and forecasting analysis
- ECTSum: earnings call transcripts with expert bullet-point summaries - for summarisation tasks
______
For sentiment analysis or financial specific natural language extraction we would employ readily available NLP tools - pre-trained domain models or enterprise grade NLP pipelines.

## NLP tools for finance
- FinBERT: specialised BERT model fine-tuned for financial sentiment analysis, built on top of Hugging Faces transformers. It outputs positive, negative and neutral with corresponding probabilities.
- Spark NLP & Spark OCR: enterprise-grade NLP library optimized for scalability on Spark. Includes pipelines for tokenization, NER, sentiment, document classification, plus OCR integration
- deepset Haystack: open-source framework for building search, QA, summarization, and RAG pipelines. Integrates with large language models via Hugging Face, OpenAI, Cohere
- Contextual AI: enterprise-focused platform for Retrieval-Augmented Generation (RAG), enabling specialised agents for banking and finance
- Cohere: large language model APIs for classification, summarization, and financial applications, now with a secure “North for Banking” platform
______
For the purpose of experimentation we would first be using **MarketAux** for fresh financial news via API, and **FinBERT** for sentiment classification on said financial news.
______

# MaketAux
A free market and financial news API. Gives access to global stock market and finance news, including funds, crypto. 

Link to website and documentation: https://www.marketaux.com/

In [31]:
# import necessary libraries
import os
from dotenv import load_dotenv

# import custom 
from utils import get_marketaux_news

# load in environment variables
load_dotenv()

# autoreload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
# load in API key
api_key = os.environ.get("MARKET_AUX_API_TOKEN")

In [33]:
# get news
status, news = get_marketaux_news(["TSLA"], api_key)

In [34]:
news

{'meta': {'found': 74098, 'returned': 3, 'limit': 3, 'page': 1},
 'data': [{'uuid': 'd9ee1c1d-6561-4fe8-8bd3-2d6f69de61b1',
   'title': 'Why Are Meta Platforms, Microsoft, and Nvidia Outperforming the "Magnificent Seven" and the S&P 500?',
   'description': 'Meta Platforms, Microsoft, and Nvidia are turning AI investments into concrete results.  Artificial intelligence has helped these companies grow their profit margins.  Meta Platforms (NASDAQ: META), Microsoft (NASDAQ: MSFT), and Nvidia (NASDAQ: NVDA) are knocking on the door of all-time highs, whereas the other four "Magnificent Seven" stocks -- Amazon, Alphabet, Tesla, and Apple -- are down year to date.',
   'keywords': 'Nvidia, Microsoft, Meta Platforms, Microsoft 365, profit margins, Magnificent Seven',
   'snippet': 'The market hates uncertainty, but it also loves companies with a clear vision for growing revenue and converting capital expenditures into free cash flow. Meta,...',
   'url': 'https://finance.yahoo.com/news/why-m

_____
As one can see from above, the API responses from most of these open source API calls would usually return a title with descriptions on the news and not return full news texts. 

This does not suit our project which aims to do analysis of full news with FinBERT. Hence, we would be employing other methods to gather news.
_____